## Introduction

Aqueous solubility is a key physical property of interest in the medicinal and agrochemical industry. Low aqueous solubility of compounds can be a major problem in drug development, as more than 40% of newly developed chemicals are practically insoluble in water. For a drug to be absorbed it needs to be contained in a solution at the site of the absorption and solubility is the main parameter that influences the bioavailability of a molecule. 

As designing and approving a new drug is an expensive, nearly decade-long process, new methods for the prediction of a compound's aqueous solubility prior to its synthesis could greatly facilitate the process of drug development. Aqueous solubility is also a major factor in the development of insecticides, fungicides and herbicides, so the agrochemical industry can also greatly benefit from new methods of estimating aqueous solubility of compounds without the presence of a physical sample. 

Machine learning allows us to create models that can predict a compound's aqueous solubility straight from its molecular structure without the presence of a physical sample and without running sophisticated physical simulations.


## Methods

The entire project was completed sing Python. We used the dataset ESOL, containing the solubilities of 1144 chemical compounds and their structures in the SMILES format.

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

#installing libraries 
import numpy as np
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
import pandas as pd

#Reading dataset
comp = pd.read_csv("raw_esol.csv")
comp
comp.info()

The library rdkit was used to extract features for the model, such as LogP (known as partition coefficient), molecular weight, rotatable bonds, H-bond donor count, H-bond acceptor count, polar surface area, aromatic proportion and non-carbon proportion.

In [ ]:
!conda install -c rdkit rdkit -y
from rdkit import Chem
mol=[Chem.MolFromSmiles(drug) for drug in comp.SMILES]
len(mol)

In [3]:
#Using rdkit to extract molecule data to set parameters - cLogP, Molecular weight, Rotatable bonds, H-bond donor count, H-bond acceptor count, Polar surface area
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
def parameters(smiles, verbose=False):
    base_data = []
    for s in smiles:
        mol=Chem.MolFromSmiles(s)
        base_data.append(mol)

    data = np.arange(1,1)
    r = 0
    for m in base_data:
        Mol_LogP = Descriptors.MolLogP(m)
        Mol_Weight = Descriptors.MolWt(m)
        Rotable_Bonds = Descriptors.NumRotatableBonds(m)
        H_bond_donor = Lipinski.NumHDonors(m)
        H_bond_acceptor = Lipinski.NumHAcceptors(m)
        Polar_Surface_Area = Descriptors.TPSA(m)

        row_data = np.array([Mol_LogP,
                             Mol_Weight,
                             Rotable_Bonds,
                             H_bond_donor,
                             H_bond_acceptor,
                             Polar_Surface_Area])
        if (r==0):
            data = row_data
        else:
            data = np.vstack([data, row_data])
        r = r + 1

    column_names = ["cLogP", "Molecular weight", "Rotatable bonds", "H-bond donor count", "H-bond acceptor count", "Polar surface area"]
    df = pd.DataFrame(data=data, columns=column_names)
    return df

ModuleNotFoundError: No module named 'rdkit'

In [ ]:
# Calculating aromatic propertion
def Aro_Atoms(mol):
    aro_atoms = [mol.GetAtomWithIdx(i).GetIsAromatic() for i in range(mol.GetNumAtoms())]
    
    a_atoms = []
    for a in aro_atoms:
        if a == True:
            a_atoms.append(a)
    sum_a_atoms = sum(a_atoms)
    return sum_a_atoms

aromatic_proportion = [Aro_Atoms(element)/Descriptors.HeavyAtomCount(element) for element in mol]
aromatic_proportion = pd.DataFrame(aromatic_proportion, columns=['Aromatic proportion'])
aromatic_proportion

# Calculating non-carbon propertions
non_carbon_proportion = [Lipinski.NumHeteroatoms(element)/Lipinski.HeavyAtomCount(element) for element in mol]
non_carbon_proportion = pd.DataFrame(non_carbon_proportion, columns=['Non carbon proportion'])
non_carbon_proportion

# X and Y matrix 
X = pd.concat([descriptors, aromatic_proportion, non_carbon_proportion], axis=1)
Y = comp.iloc[:,1]

## Linear Regression

We decided to reproduce the original ESOL research paper and trained a linear regression model from the library sklearn to predict *log(solubility)* to see if we can create a machine learning model with decent predictive power basing on the features we had previously extracted.

In [4]:
#Splitting data into training and test data sets (80%-20%)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train, Y_train)
linear_Y_train_pred = linear.predict(X_train)

#Calculating RMSE (Root mean squared error - which tells you how far the regression line data points are away from actual values)
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse_linear_Y_train_pred = sqrt(mean_squared_error(Y_train, linear_Y_train_pred))

#Calculating R^2 Score 
from sklearn.metrics import r2_score
linear_Y_train_score = r2_score(Y_train, linear_Y_train_pred)

#Applying linear regression model to testing data set 
linear_Y_test_pred = linear.predict(X_test)

#Calculating RMSE
rmse_linear_Y_test_pred = sqrt(mean_squared_error(Y_test, linear_Y_test_pred))

#Calculating R^2 Score 
linear_Y_test_score = r2_score(Y_test, linear_Y_test_pred)


This resulted in a linear regression model with the intercept value of approximately $0.118$, training $RMSE$ and $R^2$ values of approximately $0.964$ and $0.788$, and testing $RMSE$ and $R^2$ values of approximately $0.993$ and $0.779$. The model had some predictive power, which inspired us to create a more sophisticated neural network model for the same purpose.
</br>
![linear_results](plot_horizontal_logS.png)

## Neural Network

We created a neural network model using keras and tensorflow.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
ann = Sequential()
ann.add(Dense(60, input_dim=8, activation='tanh'))
ann.add(Dense(40, input_dim=60, activation='tanh'))
ann.add(Dense(20, input_dim=40, activation='tanh'))
ann.add(Dense(7, input_dim=20, activation='tanh'))
ann.add(Dense(1, input_dim=7, activation='linear'))

tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
ann.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')
ann.summary

#Fitting/ tuning the model 
ann.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.10, verbose=True)

#Applying the model to the training data set 
ann_Y_train_pred = ann.predict(X_train)

#Calculating RMSE
rmse_ann_Y_train_pred = sqrt(mean_squared_error(Y_train, ann_Y_train_pred))

#Calculating R^2 Score 
ann_Y_train_score = r2_score(Y_train, ann_Y_train_pred)

#Test data set 
ann_Y_test_pred = ann.predict(X_test)

#Calculating RMSE
rmse_ann_Y_test_pred = sqrt(mean_squared_error(Y_test, ann_Y_test_pred))

#Calculating R^2 Score
ann_Y_test_score = r2_score(Y_test, ann_Y_test_pred)

#Molecule number array for ANN
mol_numbers = np.arange(1, 230, 1).reshape(229,1)

This resulted in a neural network with training $RMSE$ and $R^2$ values of approximately $0.721$ and $0.881$, and testing $RMSE$ and $R^2$ values of approximately $0.722$ and $0.881$.
</br>

## Conclusions

The neural network model resulted in lower values of root-mean-square error ($RMSE$) and values of the coefficient of determination ($R^2$) closer to 1, suggesting better predictive power. The models can be compared using this graph, showing the distribution of measured molecule solubility and predicted molecule solubility of both models.
</br>
![comparison](models_comparison.png)